In [1]:
import scvelo as scv
from IPython.display import clear_output
import matplotlib.backends.backend_pdf
from tqdm import tnrange, tqdm_notebook
import scanpy as sc
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline


scv.logging.print_version()
scv.settings.set_figure_params(
    'scvelo', dpi_save=100, dpi=80, transparent=True)
scv.settings.verbosity = 2

Running scvelo 0.1.16.dev32+c00a55e.dirty (python 3.6.6) on 2021-01-27 10:50.


In [1]:
# paths and names
data_path='G:/data/scSLAMseq/revision/'
signatures_path='G:/data/scrnaseq_signature_collection/'
figure_path='G:/data/figures/'
libraries = ['AB', 'CE', 'DF']
donors=['B2-040', 'C2-019', 'OT227', 'OT302', 'P009T', 'P013T']

# DMSO latent time heatmap

In [76]:
obs=['Stem', 'Goblet', 'MAPK_progeny', 'YAP_targets', 'Stem_Lgr5_ISC-Merlos', 'TC1', 'TC4']
from scipy.stats import zscore
from matplotlib.backends.backend_pdf import PdfPages

In [78]:
with PdfPages(figure_path+'DMSO_sigs_vs_latenttime_heatmaps.pdf') as pdf:
    for obs_ in obs:
        from scipy.interpolate import interp1d
        Y = []
        for donor in donors:
            tab=pd.read_excel(data_path+'/tables/DMSO_only_'+donor+'_ccreg_tables.xlsx')
            f = interp1d(tab['latent_time'], tab[obs_])
            y=f(np.linspace(0,1,1000))
            Y.append(y)
        Y = np.array(Y).T

        Y=zscore(Y)
        df = pd.DataFrame(Y, columns=donors)
        if True:
            n_convolve=50
            weights = np.ones(n_convolve) / n_convolve
            for o in donors:
                try:
                    df[o] = np.convolve(df[o].values, weights, mode="same")
                except:
                    print('convolve failed')
                    pass  # e.g. all-zero counts or nans cannot be convolved
        from seaborn import clustermap
        cm = clustermap(df.T, row_cluster=True, col_cluster=False, vmin=-2, vmax=2, cmap='bwr')#, **kwargs
        cm.ax_heatmap.set_xlabel('latent time');
        cm.ax_heatmap.set_title(obs_+' heatmap');
        cm.ax_heatmap.set_xticks([]);
        pdf.savefig()  # saves the current figure into a pdf page
        pl.close()